# Large Language Models

Dass es sich bei Large Language Models (LLMs) nicht um Magie oder eine starke künstliche Intelligenz (oder gar einer Artificial Generel Intelligence bzw. AGI) sondern um reine Stastistik handelt, sollte mittlerweile allen bekannt sein.

Ein LLM macht doch nichts anderes, als an den vorhandenen Text ein weiteres Wort anzuhängen. Wort für Wort. Basierend auf Statistiken, die angeben, welches Wort die höchste Wahrscheinlichkeit hat, auf das oder die vorangegangenen Worte zu folgen. **Oder?!**

Ich möchte versuchen mich der Funktionsweise von Natural Language Processing (NLP) und LLMs anzunähern. Dabei werde ich über Begriffe, wie **Embeddings**, **Tokens**, **Attention** oder **Transformer** stolpern und versuchen zu erklären was hier passiert. Ich werde auch kurz auf **Rekurrente Neuronale Netze** eingehen, spielten sie doch zum Beispiel im Google Translator in Form des **Google Neural Machine Translation (GNMT)** Systems eine Rolle.

## Ein Wort nach dem anderem?

Language Models versuchen an den gegebenen Text sinnvoll Wörter anzuhängen. Unter sinnvoll versteht man, was man von einer Person zu schreiben erwarten würde, wenn diese Person Milliarden von Text, Büchern, Internetseiten usw. gelesen und sich dann gemerkt hätte, welches Wort am häufigsten auf den vorangegangenen Text oder das vorangegangene Wort folgte.

In [1]:
%%capture
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [3]:
from huggingface_hub import hf_hub_download
import llama_cpp

In [4]:
%%capture
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [5]:
%%capture
llm = llama_cpp.Llama(model_path)

In [6]:
# Define a function to generate a response and print probabilities
def generate_response(prompt):
    output = llm(prompt)
    print("Response:", output["choices"][0]["text"])
    print("Probabilities:", output["choices"][0]["probabilities"])

In [7]:
prompt = "A small step"

In [ ]:
# Generate a response and print probabilities
generate_response(prompt)

## Rekurrente Neuronale Netze in Natural Language Processing

Im November 2016 revolutionierte Google die maschinelle Übersetzung mit der Einführung des GNMT-Systems (Google Neural Machine Translation). Dieses System nutzte eine innovative Encoder-Decoder-Architektur mit RNNs (LSTMs) und einer Aufmerksamkeitsmechanik. Durch den Einsatz von neuronalen Netzwerken verbesserte es die Übersetzungsgenauigkeit deutlich im Vergleich zum vorherigen statistischen Ansatz. Jedoch setzte Google 2017 ein noch effizienteres Modell ein - den Transformer. Dieser innovative Ansatz basierte ausschließlich auf Aufmerksamkeitsmechanismen, wie im hochgelobten Paper "Attention is All You Need" beschrieben. Vor der Einführung von GNMT verließ sich Google Translate auf herkömmliche statistische Methoden zur maschinellen Übersetzung. Diese bahnbrechenden Fortschritte haben die Präzision und Leistungsfähigkeit von Google Translate erheblich gesteigert.

Auch wenn RNNs nicht mehr State of the Art für NLP ist, lässt sich hier doch einiges veranschaulichen. Daher versuche ich im folgenden den IMDB Datensatz mit Hilfe von RNNs zu analysieren und Vorhersagen zu machen, ob eine Kritik z.B. positiv oder negativ ist.

### IMDB Datensatz


Der IMDB-Datensatz ist eine wertvolle Ressource für die natürliche Sprachverarbeitung (NLP). Er umfasst eine große Anzahl von Filmrezensionen, die jeweils mit Bewertungen versehen sind. Dieser Datensatz wird häufig in der Sentimentanalyse eingesetzt, um den sentimentalen Gehalt eines Textes zu klassifizieren, ob er positiv oder negativ ist. Mit Hilfe des IMDB-Datensatzes können Forscher und Entwickler NLP-Algorithmen trainieren und verbessern, um Texte besser zu verstehen und Bewertungen automatisch zu analysieren.

Es ist sozusagen das **Hallo Welt!** des NLP.

Im folgenden werde ich versuchen, dass Beispiel IMDB in Python umzusetzen und vorherzusagen, ob eine Kritik positiv oder negativ ist. Ich fange mit den wichtigsen Imports an.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Laden des IMDB-Datensatzes
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Vorbereiten der Daten
max_len = 200
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Erstellen des Modells
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 32, input_length=max_len),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training des Modells
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

# Vorhersagen treffen
text = "This movie was amazing!"
sequence = imdb.get_word_index()
input_text = [sequence[word.lower()] if word.lower() in sequence else 0 for word in text.split()]
input_text = pad_sequences([input_text], maxlen=max_len)
prediction = model.predict(input_text)[0][0]

# Ausgabe der Vorhersage
if prediction >= 0.5:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")